In [9]:
import socket

HOST = '127.0.0.1' 
PORT = 2912 

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
    client_socket.connect((HOST, PORT))
    print("Connected to the server.")

    while True:
        user_input = input("Enter a number (or 'exit' to quit): ")
        if user_input.lower() == 'exit':
            print("Exiting the client.")
            break

        try:
            client_socket.sendall(user_input.encode('utf-8'))
            response = client_socket.recv(1024).decode('utf-8')
            print("Server response:", response)
        except Exception as e:
            print("An error occurred:", e)
            break

Connected to the server.
Server response: The number is higher.
Server response: The number is higher.
Server response: The number is higher.
Server response: The number is lower.
Server response: Congratulations! You guessed the number.
Server response: 
Exiting the client.


In [ ]:
import socket
import random

def main():
    HOST = '127.0.0.1'
    PORT = 2912
    
    target_number = random.randint(1, 100)
    print(f"Server started. Target number is {target_number}")

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((HOST, PORT))
        server_socket.listen()
        print(f"Listening on {HOST}:{PORT}...")

        conn, addr = server_socket.accept()
        with conn:
            print(f"Connected by {addr}")
            while True:
                data = conn.recv(1024).decode('utf-8')
                if not data:
                    break

                try:
                    client_number = int(data)
                except ValueError:
                    conn.sendall(b"Error: Please send a valid number.")
                    continue

                if client_number < target_number:
                    conn.sendall(b"The number is higher.")
                elif client_number > target_number:
                    conn.sendall(b"The number is lower.")
                else:
                    conn.sendall(b"Congratulations! You guessed the number.")
                    print("Client guessed the number. Shutting down.")
                    break

if __name__ == "__main__":
    main()


In [18]:
import socket
import time

SERVER_IP = "212.182.24.27"
UDP_PORTS = [p for p in range(1, 65536) if str(p).endswith("666")]
SEQUENCE = []

print("[*] Szukanie sekwencji UDP...")
for port in UDP_PORTS:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
            s.settimeout(3)  # Zwiększony timeout
            s.sendto(b"PING", (SERVER_IP, port))
            try:
                data, _ = s.recvfrom(1024)
                print(f"[?] Port {port} odpowiedział: {data}")
                if b"PONG" in data:
                    print(f"[+] Znaleziono port w sekwencji: {port}")
                    SEQUENCE.append(port)
                    time.sleep(0.5)
            except socket.timeout:
                pass
    except Exception as e:
        print(f"[!] Błąd przy porcie {port}: {e}")

print("[*] Sekwencja UDP znaleziona:")
print(SEQUENCE)

[*] Szukanie sekwencji UDP...
[*] Sekwencja UDP znaleziona:
[]


In [12]:
import socket
import time

SERVER_IP = "127.0.0.1" 
PORT_TCP = 5001
PORT_UDP = 5002
BUFFER_SIZE = 1024
NUM_PACKETS = 1000

data = b"x" * BUFFER_SIZE

# TCP test
start_tcp = time.time()
with socket.create_connection((SERVER_IP, PORT_TCP)) as s:
    for _ in range(NUM_PACKETS):
        s.sendall(data)
end_tcp = time.time()
tcp_duration = end_tcp - start_tcp
print(f"[TCP] Czas wysyłki {NUM_PACKETS} pakietów: {tcp_duration:.4f} s")

# UDP test
udp_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
start_udp = time.time()
for _ in range(NUM_PACKETS):
    udp_sock.sendto(data, (SERVER_IP, PORT_UDP))
end_udp = time.time()
udp_sock.close()
udp_duration = end_udp - start_udp
print(f"[UDP] Czas wysyłki {NUM_PACKETS} pakietów: {udp_duration:.4f} s")

[TCP] Czas wysyłki 1000 pakietów: 0.0435 s
[UDP] Czas wysyłki 1000 pakietów: 0.0064 s


In [ ]:
import socket
import threading

HOST = "127.0.0.1" 
PORT_TCP = 5001
PORT_UDP = 5002
BUFFER_SIZE = 1024

def tcp_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT_TCP))
        s.listen(1)
        print("[TCP] Czekam na połączenie...")
        conn, addr = s.accept()
        with conn:
            print(f"[TCP] Połączono z {addr}")
            while True:
                data = conn.recv(BUFFER_SIZE)
                if not data:
                    break

def udp_server():
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.bind((HOST, PORT_UDP))
        print("[UDP] Odbieranie pakietów...")
        while True:
            data, addr = s.recvfrom(BUFFER_SIZE)
            if not data:
                break

threading.Thread(target=tcp_server, daemon=True).start()
threading.Thread(target=udp_server, daemon=True).start()



In [ ]:

UDP będzie szybsze od TCP. W testach lokalnych UDP może być nawet kilkukrotnie szybsze.

UDP nie ustanawia połączenia (brak trójfazowego handshake).
Nie ma mechanizmu potwierdzeń (ACK).
Nie kontroluje kolejności ani retransmisji — wysyła i zapomina.



TCP
Gwarantuje dostarczenie i kolejność
Wolniejszy z powodu narzutu (handshake, ACKs)
Zastosowania : WWW, FTP, e-mail

UDP
Nie gwarantuje dostarczenia ani kolejności
Prędkość: Szybszy, mniejszy narzut
Zastosowania: VoIP, gry online, transmisje video
